In [1]:
import numpy as np
import pandas as pd
import skimage.data
import matplotlib
# Reading the image  
img = skimage.data.chelsea()  
# Converting the image into gray which only has two dimensions
img = skimage.color.rgb2gray(img)

C:\Users\Jiale\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
img.shape

(300, 451)

In [3]:
len(img.shape)

2

In [4]:
# prepare filters bank
l1_filter = np.zeros((2,3,3))
# detect vertical edges
l1_filter[0, :, :] = np.array([[[-1, 0, 1],
                                 [-1, 0, 1],
                                 [-1, 0, 1]]])
# detect horizontal edges
l1_filter[1, :, :] = np.array([[[1, 1, 1],
                                 [0, 0, 0],
                                 [-1, -1, -1]]])
# If the image is RGB with 3 channels, the filter size must be (3, 3, 3=depth)

In [5]:
l1_filter.shape[1]

3

In [6]:
# define Conv function
def conv(img, conv_filter):
    # Check if number of image channels matches the filter depth.
    if len(img.shape) > 2 or len(conv_filter.shape) > 3:
        if img.shape[-1] != conv_filter.shape[-1]:
            print("Error: Number of channels in both image and filter must match.")
            sys.exit()
    # Check if filter dimensions are equal.
    if conv_filter.shape[1] != conv_filter.shape[2]:
        print('Error: Filter must be a square matrix. I.e. number of rows and columns must match.')  
        sys.exit() 
    # Check if filter diemnsions are odd.
    if conv_filter.shape[1]%2==0:
        print('Error: Filter must have an odd size. I.e. number of rows and columns must be odd.')  
        sys.exit()
        
    # An empty feature map to hold the output of convolving the filter(s) with the image.
    feature_maps = np.zeros((img.shape[0]-conv_filter.shape[1]+1,
                             img.shape[1]-conv_filter.shape[1]+1,
                             conv_filter.shape[0]))
    
    # Convolving the image by the filter(s).
    for filter_num in range(conv_filter.shape[0]):
        print('filter', filter_num + 1)
        # getting a filter from the bank.
        curr_filter = conv_filter[filter_num, :]
        """  
         Checking if there are mutliple channels for the single filter. 
         If so, then each channel will convolve the image. 
         The result of all convolutions are summed to return a single feature map. 
        """ 
        if len(curr_filter.shape) > 2:
            # Array holding the sum of all feature maps.
            conv_map = conv_(img[:,:,0], curr_filter[:,:,0])
            # Convolving each channel with the image and summing the results.
            for ch_num in range(1, curr_filter.shape[-1]):
                conv_map = conv_map + conv_(img[:,:,ch_num],
                                            curr_filter[:,:,ch_num])
        # There is just a single channel in the filter.
        else:
            conv_map = conv_(img, curr_filter)
        # Holding feature map with the current filter.
        feature_maps[:,:,filter_num] = conv_map
    # Returning all feature maps.
    return feature_maps

In [7]:
# define conv_ function
def conv_(img, conv_filter):
    filter_size = conv_filter.shape[0]
    result = np.zeros((img.shape))
    # Looping through the image to apply the convolution operation.
    for r in np.uint16(np.arange(filter_size/2,
                                img.shape[0]-filter_size/2-2)):
        for c in np.uint16(np.arange(filter_size/2, 
                                        img.shape[1]-filter_size/2-2)):
            # Getting the current region to get multiplied with the filter. 
            curr_region = img[r:r+filter_size, c:c+filter_size]
            # Element-wise multipliplication between the current region and the filter.
            curr_result = curr_region * conv_filter
            # Summing the result of multiplication. 
            conv_sum = np.sum(curr_result)
            # Saving the summation in the convolution layer feature map
            result[r, c] = conv_sum
            
    # clipping the outliers of the result matrix
    final_result = result[np.uint16(filter_size/2):result.shape[0]-np.uint16(filter_size/2),
                          np.uint16(filter_size/2):result.shape[1]-np.uint16(filter_size/2)]
    return final_result

In [8]:
# Conv layer
l1_feature_map = conv(img, l1_filter)

filter 1
filter 2


In [9]:
l1_feature_map.shape

(298, 449, 2)

In [10]:
# Define Relu function
def relu(feature_map):
    # Preparing the output of the ReLU activation function.
    relu_out = np.zeros(feature_map.shape)
    for map_num in range(feature_map.shape[-1]):
        for r in np.arange(0, feature_map.shape[0]):
            for c in np.arange(0, feature_map.shape[1]):
                relu_out[r,c,map_num] = np.max(feature_map[r,c,map_num], 0)
    return relu_out 

In [11]:
# Apply Relu layer
l1_feature_map_relu = relu(l1_feature_map)

In [12]:
l1_feature_map_relu.shape

(298, 449, 2)

In [13]:
# Define pooling function
def pooling(feature_map, size=2, stride=2):
    # Preparing the output of the pooling operation.
    pool_out = np.zeros((np.uint16((feature_map.shape[0]-size+1)/stride),
                         np.uint16((feature_map.shape[1]-size+1)/stride),
                         feature_map.shape[-1]))
    for map_num in range(feature_map.shape[-1]):
        r2 = 0
        for r in np.arange(0, feature_map.shape[0]-size-1, stride):
            c2 = 0
            for c in np.arange(0, feature_map.shape[1]-size-1, stride):
                pool_out[r2, c2, map_num] = np.max(feature_map[r:r+size,  c:c+size])
                c2 = c2 + 1
            r2 = r2 + 1
    return pool_out

In [14]:
# Max pooling layer
l1_feature_map_relu_pool = pooling(l1_feature_map_relu, 2, 2)

In [15]:
l1_feature_map_relu_pool.shape

(148, 224, 2)

In [16]:
# Stacking layers
# second conv layer
l2_filter = np.random.rand(3, 5, 5, l1_feature_map_relu_pool.shape[-1])
print("\n**Working with conv layer 2**") 
l2_feature_map = conv(l1_feature_map_relu_pool, l2_filter)
print("\n**ReLU**")
l2_feature_map_relu = relu(l2_feature_map)
print("\n**Pooling**")
l2_feature_map_relu_pool = pooling(l2_feature_map_relu, 2, 2)
print("**End of conv layer 2**\n")


**Working with conv layer 2**
filter 1
filter 2
filter 3

**ReLU**

**Pooling**
**End of conv layer 2**



In [17]:
l2_feature_map_relu_pool.shape

(71, 109, 3)

In [18]:
# Third conv layer  
l3_filter = np.random.rand(1, 7, 7, l2_feature_map_relu_pool.shape[-1])  
print("\n**Working with conv layer 3**")  
l3_feature_map = conv(l2_feature_map_relu_pool, l3_filter)  
print("\n**ReLU**")  
l3_feature_map_relu = relu(l3_feature_map)  
print("\n**Pooling**")  
l3_feature_map_relu_pool = pooling(l3_feature_map_relu, 2, 2)  
print("**End of conv layer 3**\n") 


**Working with conv layer 3**
filter 1

**ReLU**

**Pooling**
**End of conv layer 3**



In [20]:
import matplotlib.pyplot as plt

In [21]:
# Graphing results
fig0, ax0 = matplotlib.pyplot.subplots(nrows=1, ncols=1)
ax0.imshow(img).set_cmap("gray")
ax0.set_title("Input Image")
ax0.get_xaxis().set_ticks([])
ax0.get_yaxis().set_ticks([])
matplotlib.pyplot.savefig("in_img.png", bbox_inches="tight")
matplotlib.pyplot.close(fig0)

In [22]:
# Layer 1
fig1, ax1 = matplotlib.pyplot.subplots(nrows=3, ncols=2)
ax1[0, 0].imshow(l1_feature_map[:, :, 0]).set_cmap("gray")
ax1[0, 0].get_xaxis().set_ticks([])
ax1[0, 0].get_yaxis().set_ticks([])
ax1[0, 0].set_title("L1-Map1")

ax1[0, 1].imshow(l1_feature_map[:, :, 1]).set_cmap("gray")
ax1[0, 1].get_xaxis().set_ticks([])
ax1[0, 1].get_yaxis().set_ticks([])
ax1[0, 1].set_title("L1-Map2")

ax1[1, 0].imshow(l1_feature_map_relu[:, :, 0]).set_cmap("gray")
ax1[1, 0].get_xaxis().set_ticks([])
ax1[1, 0].get_yaxis().set_ticks([])
ax1[1, 0].set_title("L1-Map1ReLU")

ax1[1, 1].imshow(l1_feature_map_relu[:, :, 1]).set_cmap("gray")
ax1[1, 1].get_xaxis().set_ticks([])
ax1[1, 1].get_yaxis().set_ticks([])
ax1[1, 1].set_title("L1-Map2ReLU")

ax1[2, 0].imshow(l1_feature_map_relu_pool[:, :, 0]).set_cmap("gray")
ax1[2, 0].get_xaxis().set_ticks([])
ax1[2, 0].get_yaxis().set_ticks([])
ax1[2, 0].set_title("L1-Map1ReLUPool")

ax1[2, 1].imshow(l1_feature_map_relu_pool[:, :, 1]).set_cmap("gray")
ax1[2, 0].get_xaxis().set_ticks([])
ax1[2, 0].get_yaxis().set_ticks([])
ax1[2, 1].set_title("L1-Map2ReLUPool")

matplotlib.pyplot.savefig("L1.png", bbox_inches="tight")
matplotlib.pyplot.close(fig1)

In [23]:
# Layer 2
fig2, ax2 = matplotlib.pyplot.subplots(nrows=3, ncols=3)
ax2[0, 0].imshow(l2_feature_map[:, :, 0]).set_cmap("gray")
ax2[0, 0].get_xaxis().set_ticks([])
ax2[0, 0].get_yaxis().set_ticks([])
ax2[0, 0].set_title("L2-Map1")

ax2[0, 1].imshow(l2_feature_map[:, :, 1]).set_cmap("gray")
ax2[0, 1].get_xaxis().set_ticks([])
ax2[0, 1].get_yaxis().set_ticks([])
ax2[0, 1].set_title("L2-Map2")

ax2[0, 2].imshow(l2_feature_map[:, :, 2]).set_cmap("gray")
ax2[0, 2].get_xaxis().set_ticks([])
ax2[0, 2].get_yaxis().set_ticks([])
ax2[0, 2].set_title("L2-Map3")

ax2[1, 0].imshow(l2_feature_map_relu[:, :, 0]).set_cmap("gray")
ax2[1, 0].get_xaxis().set_ticks([])
ax2[1, 0].get_yaxis().set_ticks([])
ax2[1, 0].set_title("L2-Map1ReLU")

ax2[1, 1].imshow(l2_feature_map_relu[:, :, 1]).set_cmap("gray")
ax2[1, 1].get_xaxis().set_ticks([])
ax2[1, 1].get_yaxis().set_ticks([])
ax2[1, 1].set_title("L2-Map2ReLU")

ax2[1, 2].imshow(l2_feature_map_relu[:, :, 2]).set_cmap("gray")
ax2[1, 2].get_xaxis().set_ticks([])
ax2[1, 2].get_yaxis().set_ticks([])
ax2[1, 2].set_title("L2-Map3ReLU")

ax2[2, 0].imshow(l2_feature_map_relu_pool[:, :, 0]).set_cmap("gray")
ax2[2, 0].get_xaxis().set_ticks([])
ax2[2, 0].get_yaxis().set_ticks([])
ax2[2, 0].set_title("L2-Map1ReLUPool")

ax2[2, 1].imshow(l2_feature_map_relu_pool[:, :, 1]).set_cmap("gray")
ax2[2, 1].get_xaxis().set_ticks([])
ax2[2, 1].get_yaxis().set_ticks([])
ax2[2, 1].set_title("L2-Map2ReLUPool")

ax2[2, 2].imshow(l2_feature_map_relu_pool[:, :, 2]).set_cmap("gray")
ax2[2, 2].get_xaxis().set_ticks([])
ax2[2, 2].get_yaxis().set_ticks([])
ax2[2, 2].set_title("L2-Map3ReLUPool")

matplotlib.pyplot.savefig("L2.png", bbox_inches="tight")
matplotlib.pyplot.close(fig2)

In [24]:
# Layer 3
fig3, ax3 = matplotlib.pyplot.subplots(nrows=1, ncols=3)
ax3[0].imshow(l3_feature_map[:, :, 0]).set_cmap("gray")
ax3[0].get_xaxis().set_ticks([])
ax3[0].get_yaxis().set_ticks([])
ax3[0].set_title("L3-Map1")

ax3[1].imshow(l3_feature_map_relu[:, :, 0]).set_cmap("gray")
ax3[1].get_xaxis().set_ticks([])
ax3[1].get_yaxis().set_ticks([])
ax3[1].set_title("L3-Map1ReLU")

ax3[2].imshow(l3_feature_map_relu_pool[:, :, 0]).set_cmap("gray")
ax3[2].get_xaxis().set_ticks([])
ax3[2].get_yaxis().set_ticks([])
ax3[2].set_title("L3-Map1ReLUPool")

matplotlib.pyplot.savefig("L3.png", bbox_inches="tight")
matplotlib.pyplot.close(fig3)